In [32]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

In [33]:
from huggingface_hub import notebook_login

notebook_login()

In [34]:
from datasets import load_dataset, Audio
#https://huggingface.co/datasets/doof-ferb/Speech-MASSIVE_vie
dataset = load_dataset("doof-ferb/Speech-MASSIVE_vie", split="train")
dataset

Dataset({
    features: ['id', 'scenario_str', 'intent_str', 'utt', 'worker_id', 'audio', 'is_transcript_reported', 'is_validated', 'speaker_id', 'speaker_sex', 'speaker_age', 'speaker_ethnicity_simple', 'speaker_country_of_birth', 'speaker_country_of_residence', 'speaker_nationality', 'speaker_first_language'],
    num_rows: 115
})

In [35]:
len(dataset)

115

In [36]:
print(dataset)

Dataset({
    features: ['id', 'scenario_str', 'intent_str', 'utt', 'worker_id', 'audio', 'is_transcript_reported', 'is_validated', 'speaker_id', 'speaker_sex', 'speaker_age', 'speaker_ethnicity_simple', 'speaker_country_of_birth', 'speaker_country_of_residence', 'speaker_nationality', 'speaker_first_language'],
    num_rows: 115
})


In [37]:
# # Calculate the number of rows for half the dataset
# half_size = len(dataset) // 16

# # Select the first half of the dataset
# dataset = dataset.select(range(half_size))

# print(dataset)

We are using just the 1/16th of the data

In [38]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [39]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)


In [40]:
tokenizer = processor.tokenizer

In [41]:
dataset[2:5]

{'id': ['10249', '3854', '12053'],
 'scenario_str': ['play', 'audio', 'recommendation'],
 'intent_str': ['play_game', 'audio_volume_mute', 'recommendation_movies'],
 'utt': ['hãy chơi một ván trivia',
  'tắt loa làm ơn',
  'các bộ phim được đánh giá cao đang chiếu cuối tuần này'],
 'worker_id': ['5', '30', '3'],
 'audio': [{'path': None,
   'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
           3.90625000e-03,  3.78417969e-03,  3.93676758e-03]),
   'sampling_rate': 16000},
  {'path': None,
   'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00283813,
          -0.00244141, -0.00256348]),
   'sampling_rate': 16000},
  {'path': None,
   'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          -3.05175781e-05, -3.05175781e-05, -6.10351562e-05]),
   'sampling_rate': 16000}],
 'is_transcript_reported': [False, False, False],
 'is_validated': [True, True, True],
 'speaker_id': ['5e25be7c5514e680ef436338',
  '657c8d982832

Let's normalize the dataset, create a column called "normalized_text"

In [42]:
def extract_all_chars(batch):
    all_text = " ".join(batch["utt"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [43]:
dataset_vocab - tokenizer_vocab

{' ',
 'à',
 'á',
 'â',
 'ã',
 'è',
 'ì',
 'í',
 'ò',
 'ó',
 'ô',
 'ù',
 'ú',
 'ý',
 'ă',
 'đ',
 'ĩ',
 'ơ',
 'ư',
 '̀',
 '́',
 '̃',
 '̉',
 '̣',
 'ạ',
 'ả',
 'ấ',
 'ầ',
 'ẩ',
 'ậ',
 'ắ',
 'ằ',
 'ặ',
 'ẹ',
 'ẽ',
 'ế',
 'ề',
 'ể',
 'ễ',
 'ệ',
 'ỉ',
 'ị',
 'ọ',
 'ỏ',
 'ố',
 'ổ',
 'ỗ',
 'ộ',
 'ớ',
 'ờ',
 'ở',
 'ợ',
 'ụ',
 'ủ',
 'ứ',
 'ừ',
 'ử',
 'ữ',
 'ự',
 'ỷ',
 'ỹ'}

In [44]:
import re

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation (except apostrophes)
    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['utt'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])

{'id': ['10249', '3854', '12053'], 'scenario_str': ['play', 'audio', 'recommendation'], 'intent_str': ['play_game', 'audio_volume_mute', 'recommendation_movies'], 'utt': ['hãy chơi một ván trivia', 'tắt loa làm ơn', 'các bộ phim được đánh giá cao đang chiếu cuối tuần này'], 'worker_id': ['5', '30', '3'], 'audio': [{'path': None, 'array': array([-3.05175781e-05,  0.00000000e+00, -3.05175781e-05, ...,
        3.90625000e-03,  3.78417969e-03,  3.93676758e-03]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00283813,
       -0.00244141, -0.00256348]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.05175781e-05, -3.05175781e-05, -6.10351562e-05]), 'sampling_rate': 16000}], 'is_transcript_reported': [False, False, False], 'is_validated': [True, True, True], 'speaker_id': ['5e25be7c5514e680ef436338', '657c8d982832af573ef2c039', '659ea35db097ea3c414b04c0'], '

In [45]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [46]:
dataset_vocab - tokenizer_vocab

{' ',
 'à',
 'á',
 'â',
 'ã',
 'è',
 'ì',
 'í',
 'ò',
 'ó',
 'ô',
 'ù',
 'ú',
 'ý',
 'ă',
 'đ',
 'ĩ',
 'ơ',
 'ư',
 'ạ',
 'ả',
 'ấ',
 'ầ',
 'ẩ',
 'ậ',
 'ắ',
 'ằ',
 'ặ',
 'ẹ',
 'ẽ',
 'ế',
 'ề',
 'ể',
 'ễ',
 'ệ',
 'ỉ',
 'ị',
 'ọ',
 'ỏ',
 'ố',
 'ổ',
 'ỗ',
 'ộ',
 'ớ',
 'ờ',
 'ở',
 'ợ',
 'ụ',
 'ủ',
 'ứ',
 'ừ',
 'ử',
 'ữ',
 'ự',
 'ỷ',
 'ỹ'}

In [47]:
# replacements = [
#     ("â", "a"),  # Long a
#     ("ç", "ch"),  # Ch as in "chair"
#     ("ğ", "gh"),  # Silent g or slight elongation of the preceding vowel
#     ("ı", "i"),   # Dotless i
#     ("î", "i"),   # Long i
#     ("ö", "oe"),  # Similar to German ö
#     ("ş", "sh"),  # Sh as in "shoe"
#     ("ü", "ue"),  # Similar to German ü
#     ("û", "u"),   # Long u
# ]

# def cleanup_text(inputs):
#     for src, dst in replacements:
#         inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
#     return inputs

# dataset = dataset.map(cleanup_text)

In [48]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [49]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [50]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [51]:
processed_example["speaker_embeddings"].shape

(512,)

In [52]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [53]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

115

In [54]:
dataset = dataset.train_test_split(test_size=0.1)

In [55]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [56]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [57]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

In [58]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [59]:
pip install -U transformers

In [60]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.51.3
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [61]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_vie",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    per_device_eval_batch_size=2,
    logging_steps=25,
    report_to=["tensorboard"],
    push_to_hub=True,
    hub_model_id="mialin111/speecht5-finetuned-vie",  # 👈 sửa theo username Hugging Face của bạn
)


In [62]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)


<ipython-input-62-3e9fd9c3f140>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [63]:
trainer.train()

Step,Training Loss
25,0.761900
50,0.562200
75,0.463300
100,0.411500
125,0.388900
150,0.370200
175,0.357700
200,0.336800
225,0.324600
250,0.312900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.35800135803222655, metrics={'train_runtime': 611.3572, 'train_samples_per_second': 26.171, 'train_steps_per_second': 0.818, 'total_flos': 580192890146496.0, 'train_loss': 0.35800135803222655, 'epoch': 125.0})

In [64]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mialin111/speecht5-finetuned-vie/commit/6965edae86d24a7b6c9e6a4adc5c0e93e3aabd79', commit_message='End of training', commit_description='', oid='6965edae86d24a7b6c9e6a4adc5c0e93e3aabd79', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mialin111/speecht5-finetuned-vie', endpoint='https://huggingface.co', repo_type='model', repo_id='mialin111/speecht5-finetuned-vie'), pr_revision=None, pr_num=None)

# Inference

In [65]:
model = SpeechT5ForTextToSpeech.from_pretrained("mialin111/speecht5-finetuned-vie")


config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [66]:
example = dataset["test"][0]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [67]:
text = "Xin chào, tôi là Linh"

In [68]:
import re

number_words = {
    0: "không", 1: "một", 2: "hai", 3: "ba", 4: "bốn", 5: "năm", 6: "sáu", 7: "bảy", 8: "tám", 9: "chín",
    10: "mười", 20: "hai mươi", 30: "ba mươi", 40: "bốn mươi", 50: "năm mươi", 60: "sáu mươi",
    70: "bảy mươi", 80: "tám mươi", 90: "chín mươi"
}

def number_to_words(number):
    if number < 11:
        return number_words[number]
    elif number < 20:
        return "mười " + number_words[number % 10]
    elif number < 100:
        tens, unit = divmod(number, 10)
        text = number_words[tens * 10]
        if unit == 1:
            text += " mốt"
        elif unit == 5:
            text += " lăm"
        elif unit != 0:
            text += " " + number_words[unit]
        return text
    else:
        return str(number)  # bạn có thể mở rộng cho >100 nếu cần

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_words(number)

    result = re.sub(r'\b\d+\b', replace, text)
    return result


In [69]:
replacements = [
    # Đơn vị đo lường
    ("kg", "kilôgam"),
    ("g", "gam"),
    ("mg", "miligam"),
    ("km", "kilômét"),
    ("cm", "xentimét"),
    ("mm", "milimét"),
    ("m", "mét"),

    # Đơn vị tiền tệ
    ("$", "đô la"),
    ("đ", "đồng"),
    ("vnd", "đồng"),
    ("VNĐ", "đồng"),

    # Ký hiệu và dấu câu
    ("%", "phần trăm"),
    ("&", "và"),
    (",", ""),
    (".", ""),
    (":", ""),
    (";", ""),
    ("-", " "),
    ("/", " trên "),
    ("+", " cộng "),
    ("=", " bằng "),

    # Viết tắt phổ biến
    ("TP.", "thành phố "),
    ("Q.", "quận "),
    ("P.", "phường "),
    ("ĐT", "điện thoại "),
    ("sdt", "số điện thoại "),
]


In [70]:
# Function to clean up text using the replacement pairs
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [71]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'xin chào tôi là linh'

In [72]:
inputs = processor(text=final_text, return_tensors="pt")

In [78]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier

# Load mô hình speaker encoder
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", run_opts={"device": "cpu"})

# Đọc file WAV
signal, fs = torchaudio.load("/content/speaker_vie.wav")

# Tạo speaker embedding
speaker_embeddings = classifier.encode_batch(signal).squeeze(0)


In [79]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

In [80]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
# Save the audio to a file (e.g., 'output.wav')
sf.write('output.wav', speech.numpy(), 16000)